In [4]:
import cv2
import numpy as np

In [5]:
from pathlib import Path
from dataclasses import dataclass, field
from typing import ClassVar, Literal
from os.path import exists

@dataclass(slots=True)
class InputData:
	DESTINATION_DIRECTORY: ClassVar[Path] = Path('data')
	
	data_file: Path
	base_directory: Path = field(init=False)
	
	start_frame: int = field(init=False)
	end_frame: int = field(init=False)
	step: int = field(init=False, default=1)
	
	def __post_init__(self) -> None:
		self.base_directory: Path = InputData.DESTINATION_DIRECTORY / Path(self.data_file.stem).stem
		if not exists(self.base_directory):
			import tarfile
			with tarfile.open(self.data_file, 'r') as tar:
				tar.extractall(InputData.DESTINATION_DIRECTORY)

		with open(self.base_directory / Path('temporalROI.txt'), 'r') as roi_file:
			self.start_frame, self.end_frame = map(int, roi_file.read().split())
			if not self.start_frame < self.end_frame:
				raise ValueError('Start frame must be less than end frame!')
			
			if not (self.start_frame + self.step) < self.end_frame:
				raise ValueError('Step must be less than end frame!')
	
	def __getitem__(self, config: tuple[int, Literal['ground_truth', 'input']]) -> np.ndarray:
		frame, mode = config
		if frame < self.start_frame or frame > self.end_frame:
			raise ValueError(f'Frame {frame} is out of range [{self.start_frame}, {self.end_frame}]')
		
		match mode:
			case 'input':
				directory, short, extension = 'input', 'in', 'jpg'
			
			case 'ground_truth':
				directory, short, extension = 'groundtruth', 'gt', 'png'
			
			case _:
				raise ValueError(f'Unknown mode: {mode}!')
		
		return cv2.imread(str(self.base_directory / Path(f'{directory}/{short}{frame:06d}.{extension}')), cv2.IMREAD_GRAYSCALE)
	
	def get_image_dimensions(self) -> tuple[int, int]:
		return cv2.imread(str(self.base_directory / Path(f'input/in{self.start_frame:06d}.jpg')), cv2.IMREAD_GRAYSCALE).shape[:2]

In [6]:
pedestrian = InputData(Path('pedestrian.tar.xz'))
highway = InputData(Path('highway.tar.xz'))
office = InputData(Path('office.tar.xz'))

In [7]:
from abc import ABC, abstractmethod

@dataclass(slots=True)
class UpdateMethod(ABC):
	
	@abstractmethod
	def update(self, previous_background: np.ndarray, current_background: np.ndarray) -> np.ndarray:
		pass
	

@dataclass(slots=True)
class ConservativeUpdate(UpdateMethod):
	BACKGROUND_BRIGHTNESS: ClassVar[int] = 0
	
	def update(self, previous_background: np.ndarray, current_background: np.ndarray) -> np.ndarray:
		mask = (previous_background == ConservativeUpdate.BACKGROUND_BRIGHTNESS)
		previous_background[mask] = current_background[mask]
		return previous_background.copy()

@dataclass(slots=True)
class LiberalUpdate(UpdateMethod):
	
	def update(self, previous_background: np.ndarray, current_background: np.ndarray) -> np.ndarray:
		return current_background.copy()

In [8]:
from collections.abc import Generator
from typing import Callable, Iterable


@dataclass(slots=True)
class Segmentation(ABC):
	data: InputData
	callbacks: Iterable[Callable]
	
	update_method_name: Literal['conservative', 'liberal'] = field(default='liberal', kw_only=True)
	update_method: UpdateMethod = field(init=False)
	
	def __post_init__(self) -> None:
		if not self.update_method_name in ('conservative', 'liberal'):
			raise ValueError('Unknown update method!')

		match self.update_method_name:
			case 'conservative':
				self.update_method = ConservativeUpdate()

			case 'liberal':
				self.update_method = LiberalUpdate()
	
	@abstractmethod
	def compile(self) -> Generator[np.ndarray, None, None]:
		pass

In [9]:
@dataclass(slots=True)
class MetricsCallback:
	BACKGROUND_BRIGHTNESS: ClassVar[int] = 0
	FIRST_PLAN_BRIGHTNESS: ClassVar[int] = 255
	
	true_positive: int = field(default=0, init=False)
	true_negative: int = field(default=0, init=False)
	false_positive: int = field(default=0, init=False)
	false_negative: int = field(default=0, init=False)
	
	def __call__(self, image: np.ndarray, ground_truth: np.ndarray, **kwargs) -> None:
		first_plan = MetricsCallback.FIRST_PLAN_BRIGHTNESS
		background = MetricsCallback.BACKGROUND_BRIGHTNESS
		
		true_positive_matrix: np.ndarray = np.logical_and((image == first_plan),
														  (ground_truth == first_plan))
		true_negative_matrix: np.ndarray = np.logical_and((image == background),
														  (ground_truth == background))
		false_positive_matrix: np.ndarray = np.logical_and((image == first_plan),
														   (ground_truth == background))
		false_negative_matrix: np.ndarray = np.logical_and((image == background),
														   (ground_truth == first_plan))

		self.true_positive += np.sum(true_positive_matrix)
		self.true_negative += np.sum(true_negative_matrix)
		self.false_positive += np.sum(false_positive_matrix)
		self.false_negative += np.sum(false_negative_matrix)
		self.print_metrics()
		
	def precision(self) -> float:
		if self.true_positive + self.false_positive == 0:
			return 0.0
		return self.true_positive / (self.true_positive + self.false_positive)
	
	def recall(self) -> float:
		if self.true_positive + self.false_negative == 0:
			return 0.0
		return self.true_positive / (self.true_positive + self.false_negative)
	
	def f1_score(self) -> float:
		precision = self.precision()
		recall = self.recall()
		if precision + recall == 0:
			return 0.0
		return 2 * precision * recall / (precision + recall)
	
	def print_metrics(self) -> None:
		precision = self.precision()
		recall = self.recall()
		f1_score = self.f1_score()

		print(f'{precision=:.4f}, {recall=:.4f}, {f1_score=:.4f}')

In [10]:
from time import perf_counter

@dataclass(slots=True)
class TimeCallback:
	max_time: float = field(default=float('-inf'), init=False)
	min_time: float = field(default=float('inf'), init=False)
	average_time: float = field(default=0.0, init=False)
	current_time: float = field(default_factory=perf_counter, init=False)
	frame_counter: int = field(default=0, init=False)
	
	def __call__(self, **kwargs) -> None:
		self.current_time = perf_counter() - self.current_time
		self.max_time = max(self.max_time, self.current_time)
		self.min_time = min(self.min_time, self.current_time)
		self.average_time = (self.average_time * self.frame_counter + self.current_time) / (self.frame_counter + 1)
			
		self.print_time()
		self.frame_counter += 1
		self.current_time = perf_counter()
	
	def print_time(self) -> None:
		max_time = self.max_time
		min_time = self.min_time
		average_time = self.average_time
		current_time = self.current_time

		print(f'{max_time=:.4f}, {min_time=:.4f}, {average_time=:.4f}, {current_time=:.4f}')

In [11]:
from typing import Any


@dataclass(slots=True)
class BuforSegmentation(Segmentation, ABC):
	bufor_size: int = field(default=60)
	bufor: np.ndarray = field(init=False)
	counter: int = field(default=0, init=False)
	
	def __post_init__(self) -> None:
		super(BuforSegmentation, self).__post_init__()
		if not self.bufor_size < ((self.data.end_frame - self.data.start_frame) / self.data.step + 1):
			raise ValueError('Bufor size is too big!')
		
		self.bufor = np.zeros((*self.data.get_image_dimensions(), self.bufor_size), dtype=np.uint8)
		for i in range(self.bufor_size):
			self.bufor[:, :, i] = self.data[self.data.start_frame + i * self.data.step, 'input']
	
	def update_bufor(self, image: np.ndarray) -> None:
		self.bufor[:, :, self.counter] = image
		self.counter = (self.counter + 1) % self.bufor_size
	
	def compile(self) -> Generator[np.ndarray, None, None]:
		previous_image: np.ndarray = np.full(self.data.get_image_dimensions(), 0, dtype=np.uint8)
		
		for index in range(self.data.start_frame, self.data.end_frame, self.data.step):
			image = self.data[index, 'input']
			ground_truth = self.data[index, 'ground_truth']

			ground_truth: np.ndarray = cv2.threshold(ground_truth, 165, 255, cv2.THRESH_BINARY)[1]
			
			self.update_bufor(image)
			new_background = self.process_bufor(self.bufor)

			difference: np.ndarray = cv2.absdiff(image, new_background)
			binary: np.ndarray = cv2.threshold(difference, 18, 255, cv2.THRESH_BINARY)[1]
			median: np.ndarray = cv2.medianBlur(binary, 3)
			image: np.ndarray = cv2.morphologyEx(median, cv2.MORPH_OPEN, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3)))

			image_after = self.update_method.update(previous_image, image)

			current_state: dict[str, Any] = dict(globals(), **locals())
			current_state['seg_self'] = current_state.pop('self')
			for callback in self.callbacks:
				callback(**current_state)

			previous_image = image
			yield image_after
			
	@abstractmethod
	def process_bufor(self, bufor: np.ndarray) -> np.ndarray:
		pass

In [12]:
@dataclass(slots=True)
class AverageBuforSegmentation(BuforSegmentation):

	def process_bufor(self, bufor: np.ndarray) -> np.ndarray:
		return np.mean(bufor, axis=2).astype(np.uint8)

In [13]:
@dataclass(slots=True)
class MedianBuforSegmentation(BuforSegmentation):

    def process_bufor(self, bufor: np.ndarray) -> np.ndarray:
        return np.median(bufor, axis=2).astype(np.uint8)

In [14]:
def show_images(segmentation: Segmentation, /, duration: int = 10, title: str = 'segmentation') -> None:
	for image in segmentation.compile():
		cv2.imshow(title, image)
		cv2.waitKey(duration)
	
	cv2.destroyAllWindows()

In [15]:
show_images(AverageBuforSegmentation(pedestrian, [MetricsCallback(), TimeCallback()]))

precision=0.0000, recall=0.0000, f1_score=0.0000
max_time=0.0770, min_time=0.0770, average_time=0.0770, current_time=0.0770
precision=0.0000, recall=0.0000, f1_score=0.0000
max_time=0.0770, min_time=0.0392, average_time=0.0581, current_time=0.0392
precision=0.0000, recall=0.0000, f1_score=0.0000
max_time=0.0770, min_time=0.0312, average_time=0.0491, current_time=0.0312
precision=0.0000, recall=0.0000, f1_score=0.0000
max_time=0.0770, min_time=0.0211, average_time=0.0421, current_time=0.0211
precision=0.0041, recall=0.2715, f1_score=0.0082
max_time=0.0770, min_time=0.0211, average_time=0.0398, current_time=0.0305
precision=0.0140, recall=0.4691, f1_score=0.0272
max_time=0.0770, min_time=0.0211, average_time=0.0384, current_time=0.0311
precision=0.0321, recall=0.5101, f1_score=0.0605
max_time=0.0770, min_time=0.0169, average_time=0.0353, current_time=0.0169
precision=0.0634, recall=0.6355, f1_score=0.1153
max_time=0.0770, min_time=0.0169, average_time=0.0349, current_time=0.0325
precisio

In [16]:
show_images(MedianBuforSegmentation(pedestrian, [MetricsCallback(), TimeCallback()]))

precision=0.0000, recall=0.0000, f1_score=0.0000
max_time=0.0864, min_time=0.0864, average_time=0.0864, current_time=0.0864
precision=0.0000, recall=0.0000, f1_score=0.0000
max_time=0.0912, min_time=0.0864, average_time=0.0888, current_time=0.0912
precision=0.2857, recall=0.1538, f1_score=0.2000
max_time=0.0912, min_time=0.0627, average_time=0.0801, current_time=0.0627
precision=0.3333, recall=0.1538, f1_score=0.2105
max_time=0.0912, min_time=0.0627, average_time=0.0801, current_time=0.0800
precision=0.7571, recall=0.3510, f1_score=0.4796
max_time=0.0912, min_time=0.0612, average_time=0.0763, current_time=0.0612
precision=0.9133, recall=0.5028, f1_score=0.6486
max_time=0.0912, min_time=0.0566, average_time=0.0730, current_time=0.0566
precision=0.9558, recall=0.5360, f1_score=0.6869
max_time=0.0912, min_time=0.0566, average_time=0.0741, current_time=0.0804
precision=0.9612, recall=0.6465, f1_score=0.7730
max_time=0.0912, min_time=0.0566, average_time=0.0744, current_time=0.0767
precisio

KeyboardInterrupt: 

In [17]:
show_images(AverageBuforSegmentation(highway, [MetricsCallback(), TimeCallback()]))

precision=0.3990, recall=0.6667, f1_score=0.4993
max_time=0.1173, min_time=0.1173, average_time=0.1173, current_time=0.1173
precision=0.3247, recall=0.6375, f1_score=0.4303
max_time=0.1173, min_time=0.0279, average_time=0.0726, current_time=0.0279
precision=0.2921, recall=0.6747, f1_score=0.4077
max_time=0.1173, min_time=0.0279, average_time=0.0586, current_time=0.0308
precision=0.2791, recall=0.7036, f1_score=0.3997
max_time=0.1173, min_time=0.0279, average_time=0.0559, current_time=0.0475
precision=0.2781, recall=0.7292, f1_score=0.4026
max_time=0.1173, min_time=0.0279, average_time=0.0514, current_time=0.0338
precision=0.2789, recall=0.7488, f1_score=0.4064
max_time=0.1173, min_time=0.0279, average_time=0.0495, current_time=0.0395
precision=0.2807, recall=0.7633, f1_score=0.4105
max_time=0.1173, min_time=0.0279, average_time=0.0481, current_time=0.0397
precision=0.2888, recall=0.7748, f1_score=0.4208
max_time=0.1173, min_time=0.0279, average_time=0.0467, current_time=0.0375
precisio

KeyboardInterrupt: 

In [ ]:
show_images(MedianBuforSegmentation(highway, [MetricsCallback(), TimeCallback()]))

In [ ]:
show_images(AverageBuforSegmentation(office, [MetricsCallback(), TimeCallback()]))

In [ ]:
show_images(MedianBuforSegmentation(office, [MetricsCallback(), TimeCallback()]))

In [18]:
@dataclass(slots=True)
class ApproximateBuforSegmentation(Segmentation, ABC):
	def compile(self) -> Generator[np.ndarray, None, None]:
		previous_background: np.ndarray = self.data[self.data.start_frame, 'input']
		previous_image: np.ndarray = np.full(self.data.get_image_dimensions(), 0, dtype=np.uint8)
		
		for index in range(self.data.start_frame + self.data.step, self.data.end_frame, self.data.step):
			image = self.data[index, 'input']
			ground_truth = self.data[index, 'ground_truth']
			
			ground_truth: np.ndarray = cv2.threshold(ground_truth, 165, 255, cv2.THRESH_BINARY)[1]

			new_background = self.compute_next_background(image, previous_background)

			difference: np.ndarray = cv2.absdiff(image, new_background)
			binary: np.ndarray = cv2.threshold(difference, 18, 255, cv2.THRESH_BINARY)[1]
			median: np.ndarray = cv2.medianBlur(binary, 3)
			image: np.ndarray = cv2.morphologyEx(median, cv2.MORPH_OPEN, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3)))

			image_after = self.update_method.update(previous_image, image)

			current_state: dict[str, Any] = dict(globals(), **locals())
			current_state['seg_self'] = current_state.pop('self')
			for callback in self.callbacks:
				callback(**current_state)

			previous_image = image
			yield image_after
	
	@abstractmethod
	def compute_next_background(self, image: np.ndarray, previous_background: np.ndarray) -> np.ndarray:
		pass

In [19]:
@dataclass(slots=True)
class AverageApproximateBuforSegmentation(ApproximateBuforSegmentation):
	weight_param: float = field(default=0.01)

	def compute_next_background(self, image: np.ndarray, previous_background: np.ndarray) -> np.ndarray:
		return (self.weight_param * image.astype(np.float64) + (1 - self.weight_param) * previous_background.astype(np.float64)).astype(np.uint8).copy()

In [20]:
@dataclass(slots=True)
class MedianApproximateBuforSegmentation(ApproximateBuforSegmentation):

	def compute_next_background(self, image: np.ndarray, previous_background: np.ndarray) -> np.ndarray:
		return (previous_background + (previous_background < image) - (previous_background > image)).copy()

In [21]:
show_images(AverageApproximateBuforSegmentation(pedestrian, [MetricsCallback(), TimeCallback()], weight_param=0.03))

precision=0.0000, recall=0.0000, f1_score=0.0000
max_time=0.0046, min_time=0.0046, average_time=0.0046, current_time=0.0046
precision=0.0000, recall=0.0000, f1_score=0.0000
max_time=0.0418, min_time=0.0046, average_time=0.0232, current_time=0.0418
precision=0.0000, recall=0.0000, f1_score=0.0000
max_time=0.0418, min_time=0.0046, average_time=0.0217, current_time=0.0187
precision=0.8444, recall=0.2517, f1_score=0.3878
max_time=0.0418, min_time=0.0046, average_time=0.0203, current_time=0.0160
precision=0.9581, recall=0.4494, f1_score=0.6119
max_time=0.0418, min_time=0.0046, average_time=0.0190, current_time=0.0141
precision=0.9817, recall=0.4842, f1_score=0.6486
max_time=0.0418, min_time=0.0046, average_time=0.0189, current_time=0.0185
precision=0.9752, recall=0.5973, f1_score=0.7409
max_time=0.0418, min_time=0.0046, average_time=0.0205, current_time=0.0296
precision=0.9855, recall=0.6163, f1_score=0.7584
max_time=0.0418, min_time=0.0046, average_time=0.0198, current_time=0.0152
precisio

In [ ]:
show_images(AverageApproximateBuforSegmentation(pedestrian, [MetricsCallback(), TimeCallback()], weight_param=0.05))

In [ ]:
show_images(AverageApproximateBuforSegmentation(pedestrian, [MetricsCallback(), TimeCallback()]))

In [22]:
show_images(MedianApproximateBuforSegmentation(pedestrian, [MetricsCallback(), TimeCallback()]))

precision=0.0000, recall=0.0000, f1_score=0.0000
max_time=0.0025, min_time=0.0025, average_time=0.0025, current_time=0.0025
precision=0.0000, recall=0.0000, f1_score=0.0000
max_time=0.0401, min_time=0.0025, average_time=0.0213, current_time=0.0401
precision=0.0000, recall=0.0000, f1_score=0.0000
max_time=0.0401, min_time=0.0025, average_time=0.0195, current_time=0.0158
precision=0.8409, recall=0.2450, f1_score=0.3795
max_time=0.0401, min_time=0.0025, average_time=0.0186, current_time=0.0161
precision=0.9578, recall=0.4466, f1_score=0.6092
max_time=0.0401, min_time=0.0025, average_time=0.0184, current_time=0.0175
precision=0.9816, recall=0.4797, f1_score=0.6445
max_time=0.0401, min_time=0.0025, average_time=0.0180, current_time=0.0160
precision=0.9750, recall=0.5919, f1_score=0.7366
max_time=0.0401, min_time=0.0025, average_time=0.0177, current_time=0.0161
precision=0.9854, recall=0.6105, f1_score=0.7539
max_time=0.0401, min_time=0.0025, average_time=0.0174, current_time=0.0152
precisio

KeyboardInterrupt: 

In [ ]:
show_images(AverageApproximateBuforSegmentation(highway, [MetricsCallback(), TimeCallback()]))

In [ ]:
show_images(MedianApproximateBuforSegmentation(highway, [MetricsCallback(), TimeCallback()]))

In [ ]:
show_images(AverageApproximateBuforSegmentation(office, [MetricsCallback(), TimeCallback()]))

In [ ]:
show_images(MedianApproximateBuforSegmentation(office, [MetricsCallback(), TimeCallback()]))

In [23]:
show_images(MedianApproximateBuforSegmentation(pedestrian, [MetricsCallback(), TimeCallback()], update_method_name='conservative'))

precision=0.0000, recall=0.0000, f1_score=0.0000
max_time=0.0022, min_time=0.0022, average_time=0.0022, current_time=0.0022
precision=0.0000, recall=0.0000, f1_score=0.0000
max_time=0.0222, min_time=0.0022, average_time=0.0122, current_time=0.0222
precision=0.0000, recall=0.0000, f1_score=0.0000
max_time=0.0222, min_time=0.0022, average_time=0.0133, current_time=0.0156
precision=0.8409, recall=0.2450, f1_score=0.3795
max_time=0.0222, min_time=0.0022, average_time=0.0144, current_time=0.0178
precision=0.9578, recall=0.4466, f1_score=0.6092
max_time=0.0222, min_time=0.0022, average_time=0.0144, current_time=0.0141
precision=0.9816, recall=0.4797, f1_score=0.6445
max_time=0.0222, min_time=0.0022, average_time=0.0146, current_time=0.0158
precision=0.9750, recall=0.5919, f1_score=0.7366
max_time=0.0222, min_time=0.0022, average_time=0.0147, current_time=0.0153
precision=0.9854, recall=0.6105, f1_score=0.7539
max_time=0.0222, min_time=0.0022, average_time=0.0148, current_time=0.0153
precisio

KeyboardInterrupt: 

In [ ]:
show_images(MedianApproximateBuforSegmentation(highway, [MetricsCallback(), TimeCallback()], update_method_name='conservative'))

In [ ]:
show_images(MedianApproximateBuforSegmentation(office, [MetricsCallback(), TimeCallback()], update_method_name='conservative'))

In [24]:
@dataclass(slots=True)
class MixtureOfGaussian(Segmentation):
	learning_rate: float = field(default=-1.)
	subtractor: cv2.BackgroundSubtractorMOG2 = field(default_factory=cv2.createBackgroundSubtractorMOG2)
	
	def compile(self) -> Generator[np.ndarray, None, None]:
		previous_image: np.ndarray = np.full(self.data.get_image_dimensions(), 255, dtype=np.uint8)
	
		for index in range(self.data.start_frame, self.data.end_frame, self.data.step):
			image = self.data[index, 'input']
			ground_truth = self.data[index, 'ground_truth']
	
			ground_truth: np.ndarray = cv2.threshold(ground_truth, 165, 255, cv2.THRESH_BINARY)[1]

			image = self.subtractor.apply(image, learningRate=self.learning_rate)

			image_after = self.update_method.update(previous_image, image)
	
			current_state: dict[str, Any] = dict(globals(), **locals())
			current_state['seg_self'] = current_state.pop('self')
			for callback in self.callbacks:
				callback(**current_state)
	
			previous_image = image
			yield image_after

In [25]:
show_images(MixtureOfGaussian(pedestrian, [MetricsCallback(), TimeCallback()]))

precision=0.0000, recall=0.0000, f1_score=0.0000
max_time=0.0075, min_time=0.0075, average_time=0.0075, current_time=0.0075
precision=0.0000, recall=0.0000, f1_score=0.0000
max_time=0.0207, min_time=0.0075, average_time=0.0141, current_time=0.0207
precision=0.0000, recall=0.0000, f1_score=0.0000
max_time=0.0207, min_time=0.0075, average_time=0.0148, current_time=0.0160
precision=0.0000, recall=0.0000, f1_score=0.0000
max_time=0.0207, min_time=0.0075, average_time=0.0148, current_time=0.0149
precision=0.0000, recall=0.0000, f1_score=0.0000
max_time=0.0207, min_time=0.0075, average_time=0.0155, current_time=0.0183
precision=0.5229, recall=0.2327, f1_score=0.3220
max_time=0.0300, min_time=0.0075, average_time=0.0179, current_time=0.0300
precision=0.6161, recall=0.3393, f1_score=0.4376
max_time=0.0300, min_time=0.0075, average_time=0.0177, current_time=0.0166
precision=0.7040, recall=0.3840, f1_score=0.4970
max_time=0.0312, min_time=0.0075, average_time=0.0194, current_time=0.0312
precisio

In [ ]:
show_images(MixtureOfGaussian(highway, [MetricsCallback(), TimeCallback()]))

In [ ]:
show_images(MixtureOfGaussian(office, [MetricsCallback(), TimeCallback()]))

In [ ]:
show_images(MixtureOfGaussian(pedestrian, [MetricsCallback(), TimeCallback()], subtractor=cv2.createBackgroundSubtractorMOG2(
	1000, 32., detectShadows=True
)))

In [26]:
show_images(MixtureOfGaussian(highway, [MetricsCallback(), TimeCallback()], subtractor=cv2.createBackgroundSubtractorMOG2(
	250, 16., detectShadows=False
)))

precision=0.0065, recall=1.0000, f1_score=0.0130
max_time=0.0031, min_time=0.0031, average_time=0.0031, current_time=0.0031
precision=0.0083, recall=0.8396, f1_score=0.0165
max_time=0.0552, min_time=0.0031, average_time=0.0292, current_time=0.0552
precision=0.0087, recall=0.7336, f1_score=0.0171
max_time=0.0552, min_time=0.0031, average_time=0.0282, current_time=0.0263
precision=0.0089, recall=0.6459, f1_score=0.0175
max_time=0.0552, min_time=0.0031, average_time=0.0252, current_time=0.0164
precision=0.0094, recall=0.5862, f1_score=0.0186
max_time=0.0552, min_time=0.0031, average_time=0.0233, current_time=0.0154
precision=0.0101, recall=0.5453, f1_score=0.0197
max_time=0.0552, min_time=0.0031, average_time=0.0226, current_time=0.0190
precision=0.0106, recall=0.5132, f1_score=0.0209
max_time=0.0552, min_time=0.0031, average_time=0.0235, current_time=0.0294
precision=0.0111, recall=0.4748, f1_score=0.0217
max_time=0.0552, min_time=0.0031, average_time=0.0225, current_time=0.0154
precisio

KeyboardInterrupt: 

In [ ]:
show_images(MixtureOfGaussian(office, [MetricsCallback(), TimeCallback()], subtractor=cv2.createBackgroundSubtractorMOG2(
	500, 4., detectShadows=False
)))

In [27]:
@dataclass(slots=True)
class KNearestNeighbours(Segmentation):
	learning_rate: float = field(default=-1.)
	subtractor: cv2.createBackgroundSubtractorKNN = field(default_factory=cv2.createBackgroundSubtractorKNN)

	def compile(self) -> Generator[np.ndarray, None, None]:
		previous_image: np.ndarray = np.full(self.data.get_image_dimensions(), 255, dtype=np.uint8)

		for index in range(self.data.start_frame, self.data.end_frame, self.data.step):
			image = self.data[index, 'input']
			ground_truth = self.data[index, 'ground_truth']

			ground_truth: np.ndarray = cv2.threshold(ground_truth, 165, 255, cv2.THRESH_BINARY)[1]

			image = self.subtractor.apply(image, learningRate=self.learning_rate)

			image_after = self.update_method.update(previous_image, image)

			current_state: dict[str, Any] = dict(globals(), **locals())
			current_state['seg_self'] = current_state.pop('self')
			for callback in self.callbacks:
				callback(**current_state)

			previous_image = image
			yield image_after

In [ ]:
show_images(MixtureOfGaussian(highway, [MetricsCallback(), TimeCallback()], subtractor=cv2.createBackgroundSubtractorMOG2(
	250, 400., detectShadows=False
)))

In [28]:
show_images(MixtureOfGaussian(highway, [MetricsCallback(), TimeCallback()], subtractor=cv2.createBackgroundSubtractorMOG2(
	1000, 200., detectShadows=False
)))

precision=0.0065, recall=1.0000, f1_score=0.0130
max_time=0.0026, min_time=0.0026, average_time=0.0026, current_time=0.0026
precision=0.0078, recall=0.7862, f1_score=0.0155
max_time=0.0154, min_time=0.0026, average_time=0.0090, current_time=0.0154
precision=0.0078, recall=0.6594, f1_score=0.0155
max_time=0.0160, min_time=0.0026, average_time=0.0113, current_time=0.0160
precision=0.0078, recall=0.5629, f1_score=0.0155
max_time=0.0160, min_time=0.0026, average_time=0.0125, current_time=0.0159
precision=0.0079, recall=0.4805, f1_score=0.0155
max_time=0.0177, min_time=0.0026, average_time=0.0135, current_time=0.0177
precision=0.0079, recall=0.4194, f1_score=0.0155
max_time=0.0291, min_time=0.0026, average_time=0.0161, current_time=0.0291
precision=0.0079, recall=0.3722, f1_score=0.0154
max_time=0.0291, min_time=0.0026, average_time=0.0172, current_time=0.0234
precision=0.0079, recall=0.3286, f1_score=0.0154
max_time=0.0291, min_time=0.0026, average_time=0.0170, current_time=0.0160
precisio

KeyboardInterrupt: 

In [ ]:
show_images(MixtureOfGaussian(highway, [MetricsCallback(), TimeCallback()], subtractor=cv2.createBackgroundSubtractorMOG2(
	500, 800., detectShadows=False
)))

In [ ]:
pedestrian_1000 = InputData(Path('pedestrian.tar.xz'))
pedestrian_1000.start_frame = 1_000

show_images(MedianBuforSegmentation(pedestrian_1000, [MetricsCallback(), TimeCallback()], update_method_name='conservative'))

In [29]:
from dataclasses import InitVar
from collections import deque


@dataclass(slots=True)
class ConservativeUpdateFix(UpdateMethod):
	most_frequent_removal_range: int

	image_shape: InitVar[tuple[int, ...]]
	dtype: InitVar[np.dtype]
	buffer_size: InitVar[int]
	
	buffer: deque = field(init=False)
	
	BACKGROUND: ClassVar[int] = 0
	
	def __post_init__(self, image_shape: tuple[int, ...], dtype: np.dtype, buffer_size: int = 30) -> None:
		if not buffer_size > 0:
			raise ValueError("Buffer size cannot be negative")
		
		self.buffer = deque(np.zeros((buffer_size, *image_shape), dtype=dtype), buffer_size)
	
	def update(self, previous_background: np.ndarray, current_background: np.ndarray) -> np.ndarray:
		self.buffer.append(current_background)
		
		list_buffer: list = list(self.buffer)
		all_buffer: np.ndarray = np.array(list_buffer, dtype=list_buffer[0].dtype)

		values, counts = np.unique(all_buffer, return_counts=True)
		
		top_frequent = values[: self.most_frequent_removal_range]
		
		for frequent in top_frequent:
			current_background[current_background == frequent] = ConservativeUpdateFix.BACKGROUND
		
		return current_background

segmentation = MedianBuforSegmentation(pedestrian_1000, [MetricsCallback(), TimeCallback()], update_method_name='conservative')
segmentation.update_method = ConservativeUpdateFix(5, pedestrian_1000.get_image_dimensions(), np.uint8, 30)
show_images(segmentation)

NameError: name 'pedestrian_1000' is not defined